In [22]:
import re
def preprocess_code_line(code, remove_python_common_tokens=False):
    code = code.replace('(', ' ').replace(')', ' ').replace('{', ' ').replace('}', ' ').replace('[', ' ').replace(']',
                                                                                                                  ' ').replace(
        '.', ' ').replace(':', ' ').replace(';', ' ').replace(',', ' ').replace(' _ ', '_')

    code = re.sub('``.*``', '<STR>', code)
    code = re.sub("'.*'", '<STR>', code)
    code = re.sub('".*"', '<STR>', code)
    code = re.sub('\d+', '<NUM>', code)

    code = code.split()
    code = ' '.join(code)
    if remove_python_common_tokens:
        new_code = ''
        python_common_tokens = []
        for tok in code.split():
            if tok not in [python_common_tokens]:
                new_code = new_code + tok + ' '

        return new_code.strip()

    else:
        return code.strip()

In [1]:
import pandas as pd
df = pd.read_pickle("data/jitfine/changes_complete_buggy_line_level.pkl")
df['changed_line'] = df['raw_changed_line'].apply(lambda x: preprocess_code_line(x))
df

NameError: name 'preprocess_code_line' is not defined

In [26]:
def get_id(data_path='vul-dataset', type='by_time'):
    with open(f"{data_path}/{type}/train_ids.txt") as file:
        train_id = [line.rstrip() for line in file]
    with open(f"{data_path}/{type}/test_ids.txt") as file:
        test_id = [line.rstrip() for line in file]
    return train_id, test_id

In [30]:
def generate_localization_data(type='by_time'):
    train_id, test_id = get_id(data_path='data/split_data', type=type)
    test = df.loc[df['commit_id'].isin(test_id)]
    test = test.sort_values(by=['commit_id', 'idx'])
    test = test.reset_index().drop('index', axis=1)
    print(test['commit_id'].nunique())
    test.to_pickle(f'data/jitfine/{type}/changes_complete_buggy_line_level.pkl')

generate_localization_data(type='by_time')
generate_localization_data(type='cross_project')

1227
1261


In [5]:
test = pd.read_pickle("data/jitfine/by_time/changes_complete_buggy_line_level.pkl")
test

,commit_id,idx,changed_type,label,raw_changed_line,changed_line
0,00302787b788c5ff04cb6f62aed5a74d936e86c0,0,added,1,"return GetInput(context, node, index);",return GetInput context node index
1,00302787b788c5ff04cb6f62aed5a74d936e86c0,1,deleted,0,node->inputs->data[i...,node->inputs->data index != kTfLiteOptionalTensor
2,00302787b788c5ff04cb6f62aed5a74d936e86c0,2,deleted,0,if (use_tensor) {,if use_tensor
3,00302787b788c5ff04cb6f62aed5a74d936e86c0,3,deleted,0,"return GetMutableInput(context, node, index);",return GetMutableInput context node index
4,00302787b788c5ff04cb6f62aed5a74d936e86c0,4,deleted,0,},
...,...,...,...,...,...,...
318156,fff90422d181744cd75dbf011687ee7095f02875,328,added,0,ID3D10Multithread_SetMultithreadProtec...,ID<NUM>D<NUM>Multithread_SetMultithreadProtect...
318157,fff90422d181744cd75dbf011687ee7095f02875,329,added,0,ID3D10Multithread_Release(pMultithread);,ID<NUM>D<NUM>Multithread_Release pMultithread
318158,fff90422d181744cd75dbf011687ee7095f02875,330,added,0,},
318159,fff90422d181744cd75dbf011687ee7095f02875,331,added,0,return 0;,return <NUM>


In [15]:
test[test['commit_id']=='64045a6a173e6cf74a9c30bbe21a93d105be289e']

,commit_id,idx,changed_type,label,raw_changed_line,changed_line
104183,64045a6a173e6cf74a9c30bbe21a93d105be289e,0,added,0,"static int str_in_list(const char *str, char *...",static int str_in_list const char *str char * ...
104184,64045a6a173e6cf74a9c30bbe21a93d105be289e,1,added,0,{,
104185,64045a6a173e6cf74a9c30bbe21a93d105be289e,2,added,0,\tint i;,int i
104186,64045a6a173e6cf74a9c30bbe21a93d105be289e,3,added,0,\tfor (i = 0; i < count; i++),for i = <NUM> i < count i++
104187,64045a6a173e6cf74a9c30bbe21a93d105be289e,4,added,0,"\t\tif (!strcmp(list[i], str))",if !strcmp list i str
...,...,...,...,...,...,...
104550,64045a6a173e6cf74a9c30bbe21a93d105be289e,367,added,0,\t\tif (target >= reg->offset && target < reg-...,if target >= reg->offset && target < reg->offs...
104551,64045a6a173e6cf74a9c30bbe21a93d105be289e,368,added,0,\t\t\treturn 1;,return <NUM>
104552,64045a6a173e6cf74a9c30bbe21a93d105be289e,369,added,0,\t},
104553,64045a6a173e6cf74a9c30bbe21a93d105be289e,370,added,0,\treturn 0;,return <NUM>


In [3]:
res = pd.read_csv("localization/cross_project/res_localization.csv")
res['commit_id'].nunique()

858